# 03 - Modeling

## Imports

In [661]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [662]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp

In [663]:
import sys
sys.path.append('../')

import src

In [664]:
plt.style.use('fivethirtyeight')

## Prepare data

In [665]:
df = pd.read_pickle('../data/training.pickle')

In [666]:
df.head(1)

,type,date,operation,lat,long,sex,age,ethnicity_self,ethnicity_officer,legislation,search_target,outcome,found_target,stripped,station,offense,success
observation_id,,,,,,,,,,,,,,,,,
d62c9e35-6293-45fc-aab6-706bdac1601e,Person search,2017-12-01 00:00:00+00:00,False,50.923411,-0.461108,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Offender given drugs possession warning,False,False,sussex,True,False


#### Drop metropolitan station from dataset

In [667]:
df = df[df.station!='metropolitan']

#### Select relevant columns

Requests will have the following format:
```
["observation_id": <string>,
 "Type": <string>,
 "Date": <string>,
 "Part of a policing operation": <boolean>,
 "Latitude": <float>,
 "Longitude": <float>,
 "Gender": <string>,
 "Age range": <string>,
 "Officer-defined ethnicity": <string>,
 "Legislation": <string>,
 "Object of search": <string>,
 "station": <string>]
 '''


In [668]:
supplied_columns = ['type', 'date', 'operation', 'lat', 'long', 'sex', 'age', 'ethnicity_officer', 'legislation', 'search_target', 'station']

In [670]:
X = df[supplied_columns]
y = df.success.values

### Processed Data Profile

In [140]:
profile = pp.ProfileReport(X, title='Training Data Profiling Report', explorative=False, minimal=False, pool_size=0, progress_bar=True)
profile.to_file('../reports/profiling/processed_profile.html')

NameError: name 'pp' is not defined

## Pipeline

In [671]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer, KNNImputer

In [672]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, KBinsDiscretizer, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from category_encoders.target_encoder import TargetEncoder

### Preprocessing
#### Categorical Features

##### Assume searches with missing operation field did not happen as part of policing operation.

In [673]:
# df.operation = df.operation.fillna(False)
operation_imputer = SimpleImputer(strategy='constant', fill_value=False)

##### Replace missing legislations with 'unknown', then one-hot-encode column.

In [674]:
# df.legislation = df.legislation.astype('str').fillna('unknown').astype('category')
legislation_imputer = SimpleImputer(strategy='constant', fill_value='unknown')
legislation_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
legislation_preprocessor = Pipeline(steps=[('legislation_imputer', legislation_imputer),
                                           ('legislation_encoder', legislation_encoder)])

##### One-hot-encode categorical features

In [675]:
# categoricals = X.select_dtypes(['category']).columns.tolist()
# categoricals.remove('legislation') # feature is already dealt with
# categoricals.remove('ethnicity_officer') # will not use this feature to avoid discrimination

categoricals = ['type', 'sex', 'age', 'search_target', 'station']
oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

##### Combine categorical preprocessing steps

In [676]:
cat_preprocessor = ColumnTransformer(transformers=[('operation', operation_imputer, ['operation']),
                                                   ('legislation', legislation_preprocessor, ['legislation']),
                                                   ('cat_encoder', oh_encoder, categoricals)],
                                     remainder='drop')

In [677]:
_ = cat_preprocessor.fit_transform(X[:1000], y[:1000])

In [678]:
pd.DataFrame(_).head(3)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,False,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,False,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,False,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


#### Coordinate Features

##### Fill missing coordinate data with station means

In [679]:
#coordinate_dict = X.groupby('station')[['lat', 'long']].mean().fillna(X.mean()).to_dict()
coordinate_dict = \
{'lat': {'avon-and-somerset': 51.33301534853675,
  'bedfordshire': 51.99520005035238,
  'btp': 52.04498213727202,
  'cambridgeshire': 52.401295645642094,
  'cheshire': 53.27243561712015,
  'city-of-london': 51.515283227020134,
  'cleveland': 54.581267715107955,
  'cumbria': 54.53797709350655,
  'derbyshire': 53.01032026513161,
  'devon-and-cornwall': 50.53555175443321,
  'dorset': 50.71701186632197,
  'durham': 54.68330272246215,
  'dyfed-powys': 52.06194421428571,
  'essex': 51.72007429847026,
  'gloucestershire': 51.84977152248181,
  'greater-manchester': 53.47859898644197,
  'gwent': 51.60761234256556,
  'hampshire': 50.93652569163378,
  'hertfordshire': 51.76037049985289,
  'humberside': 53.48481868431375,
  'kent': 51.374468082246494,
  'lancashire': 53.78500562755108,
  'leicestershire': 52.65751631679406,
  'lincolnshire': 53.10683554523824,
  'merseyside': 53.436241618732225,
  'metropolitan': 52.515362957924715,
  'norfolk': 52.64006103976567,
  'north-wales': 53.17569351116731,
  'north-yorkshire': 54.041700087409595,
  'northamptonshire': 52.29232862938288,
  'northumbria': 54.98786282962144,
  'nottinghamshire': 52.98302869082842,
  'south-yorkshire': 52.515362957924715,
  'staffordshire': 52.87808759739836,
  'suffolk': 52.139193867362344,
  'surrey': 51.32796300224013,
  'sussex': 50.90980622282589,
  'thames-valley': 51.69650944703028,
  'warwickshire': 52.36734886374945,
  'west-mercia': 52.413887931124385,
  'west-yorkshire': 53.77299550808837,
  'wiltshire': 51.37899916477772},
 'long': {'avon-and-somerset': -2.6959433559473505,
  'bedfordshire': -0.4249400266868079,
  'btp': -0.7715286391456624,
  'cambridgeshire': -0.04725813646789005,
  'cheshire': -2.6387643686903965,
  'city-of-london': -0.0896220290722526,
  'cleveland': -1.2457966712230222,
  'cumbria': -3.136673164935065,
  'derbyshire': -1.4773875202376467,
  'devon-and-cornwall': -4.066873729177866,
  'dorset': -2.0917077684638166,
  'durham': -1.594638016918649,
  'dyfed-powys': -4.084971542857141,
  'essex': 0.538986324416861,
  'gloucestershire': -2.188883543046356,
  'greater-manchester': -2.248505129745514,
  'gwent': -3.0290781326530665,
  'hampshire': -1.1898053439685032,
  'hertfordshire': -0.2575871883632099,
  'humberside': -0.4107479908496726,
  'kent': 0.5604151309456336,
  'lancashire': -2.7154059084062276,
  'leicestershire': -1.1627258218159886,
  'lincolnshire': -0.35191183908730256,
  'merseyside': -2.9378837346396676,
  'metropolitan': -1.3420896304429029,
  'norfolk': 1.1136251113436533,
  'north-wales': -3.5303938708385822,
  'north-yorkshire': -1.1725005511775388,
  'northamptonshire': -0.8297320040567943,
  'northumbria': -1.5853070855440516,
  'nottinghamshire': -1.1530782153432024,
  'south-yorkshire': -1.3420896304429029,
  'staffordshire': -2.0221622171637286,
  'suffolk': 1.0365897135618494,
  'surrey': -0.4413027473143175,
  'sussex': -0.16365491495299755,
  'thames-valley': -0.9660928864365886,
  'warwickshire': -1.4958054560654521,
  'west-mercia': -2.3846032813072715,
  'west-yorkshire': -1.6477957410954118,
  'wiltshire': -1.9166443269398472}}

In [680]:
def fill_coordinates_with_station_means(X):
    '''Fills missing coordinates with mean coordinates of respective station.'''
    X_ = X.copy()[['lat', 'long']]
    X_.lat = X_.lat.fillna(X.station.map(coordinate_dict['lat'])).values
    X_.long = X_.long.fillna(X.station.map(coordinate_dict['long'])).values
    return X_

In [681]:
def grid_to_category(X):
    '''Combines longitude and latitude categories into a single feature.'''
    X_ = np.array([','.join(row) for row in X.astype(int).astype(str)]).reshape(-1, 1)
    return X_

In [682]:
coordinate_imputer = FunctionTransformer(fill_coordinates_with_station_means)
coordinate_discretizer = KBinsDiscretizer(n_bins=50, encode='ordinal')
coordinate_categorizer = FunctionTransformer(grid_to_category)
coordinate_encoder = TargetEncoder(cols=None, handle_missing='value', handle_unknown='value')

In [683]:
coordinate_preprocessor = Pipeline(steps=[('fill_means', coordinate_imputer),
                                          ('discretize', coordinate_discretizer),
                                          ('categorizer', coordinate_categorizer),
                                          ('encode', coordinate_encoder),
                                         ])

#### Datetime features

In [684]:
import datetime as dt

In [685]:
def extract_datetime_features(X):
    '''Extracts features from Date column:
    - hour
    - day of the week
    - days after sample start
    - square root of days since sample start
    '''
    X_ = pd.DataFrame(index=X.index)
    X_['hour'] = X['date'].dt.hour
    X_['weekday'] = X['date'].dt.weekday
    X_['daycount'] = (X['date'].dt.date - dt.date(2017, 12, 1)).dt.days
    X_['sqrt_daycount'] = X_.daycount**0.5
    return X_

In [686]:
date_extractor = FunctionTransformer(extract_datetime_features)
date_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
date_scaler = StandardScaler()

In [687]:
date_transformer = ColumnTransformer(transformers=[('oh_encoder', date_encoder, ['hour', 'weekday']),
                                                   ('scaler', date_scaler, ['daycount', 'sqrt_daycount']),
                                                  ])

In [688]:
date_preprocessor = Pipeline(steps=[('extract', date_extractor),
                                    ('transform', date_transformer)])

#### Combine all preprocessing steps

In [689]:
feature_preprocessor = FeatureUnion(transformer_list=[('categoricals', cat_preprocessor),
                                                      ('location', coordinate_preprocessor),
                                                      ('date', date_preprocessor)])

### Model Selection

In [735]:
from sklearn.model_selection import train_test_split

In [734]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=X[['station']], shuffle=True)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [736]:
model1 = xgboost.XGBClassifier()
classifiers.append(model1)
model2 = svm.SVC()
classifiers.append(model2)
model3 = tree.DecisionTreeClassifier()
classifiers.append(model3)
model4 = RandomForestClassifier()
classifiers.append(model4)

NameError: name 'xgboost' is not defined

In [ ]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred= clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm)

### Model

In [690]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report, recall_score, confusion_matrix

In [656]:
clf = DecisionTreeClassifier()

In [691]:
clf2 = LogisticRegression()

In [622]:
clf3 = SGDClassifier(loss='log')

### Build pipeline

In [657]:
pipeline = Pipeline(steps=[('preprocess', feature_preprocessor),
                           ('clf', clf)])

In [692]:
pipeline2 = Pipeline(steps=[('preprocess', feature_preprocessor),
                            ('clf', clf2)])

In [623]:
pipeline3 = Pipeline(steps=[('preprocess', feature_preprocessor),
                            ('clf', clf3)])

## Training

In [694]:
pipeline.fit(X, y)

/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('preprocess',
                 FeatureUnion(transformer_list=[('categoricals',
                                                 ColumnTransformer(transformers=[('operation',
                                                                                  SimpleImputer(fill_value=False,
                                                                                                strategy='constant'),
                                                                                  ['operation']),
                                                                                 ('legislation',
                                                                                  Pipeline(steps=[('legislation_imputer',
                                                                                                   SimpleImputer(fill_value='unknown',
                                                                                                                 strategy='co

In [695]:
pipeline2.fit(X, y)

/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTA

Pipeline(steps=[('preprocess',
                 FeatureUnion(transformer_list=[('categoricals',
                                                 ColumnTransformer(transformers=[('operation',
                                                                                  SimpleImputer(fill_value=False,
                                                                                                strategy='constant'),
                                                                                  ['operation']),
                                                                                 ('legislation',
                                                                                  Pipeline(steps=[('legislation_imputer',
                                                                                                   SimpleImputer(fill_value='unknown',
                                                                                                                 strategy='co

In [696]:
pipeline3.fit(X, y)

/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/rubelrennfix/.virtualenvs/capstone/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('preprocess',
                 FeatureUnion(transformer_list=[('categoricals',
                                                 ColumnTransformer(transformers=[('operation',
                                                                                  SimpleImputer(fill_value=False,
                                                                                                strategy='constant'),
                                                                                  ['operation']),
                                                                                 ('legislation',
                                                                                  Pipeline(steps=[('legislation_imputer',
                                                                                                   SimpleImputer(fill_value='unknown',
                                                                                                                 strategy='co

#### Evaluate

In [697]:
def authorise_search(pipeline, X):
    '''Authorises a search whenever there is a probability greater than 10% that the search will be successful.'''
    authorise = pipeline.predict_proba(X)[:,1] > 0.1
    return authorise

In [698]:
authorised = authorise_search(pipeline, X)

In [699]:
authorised2 = authorise_search(pipeline2, X)

In [700]:
authorised3 = authorise_search(pipeline3, X)

In [701]:
pd.Series(authorised).value_counts()/len(y)

False    0.772567
True     0.227433
dtype: float64

In [702]:
pd.Series(authorised2).value_counts()/len(y)

True     0.815759
False    0.184241
dtype: float64

In [703]:
pd.Series(authorised3).value_counts()/len(y)

True     0.836779
False    0.163221
dtype: float64

In [704]:
confusion_matrix(y, authorised)

array([[238965,   8066],
       [     3,  62283]])

In [705]:
confusion_matrix(y, authorised2)

array([[ 53850, 193181],
       [  3139,  59147]])

In [706]:
confusion_matrix(y, authorised3)

array([[ 47885, 199146],
       [  2602,  59684]])

In [712]:
recall_score(y, authorised)

0.9999518350833253

In [713]:
recall_score(y, authorised2)

0.949603442186045

In [714]:
recall_score(y, authorised3)

0.9582249622708152

In [715]:
classification_report(y, authorised, output_dict=True)

{'False': {'precision': 0.9999874460178769,
  'recall': 0.9673482275503884,
  'f1-score': 0.9833970851791876,
  'support': 247031},
 'True': {'precision': 0.8853430752391648,
  'recall': 0.9999518350833253,
  'f1-score': 0.9391638707731745,
  'support': 62286},
 'accuracy': 0.9739134932771235,
 'macro avg': {'precision': 0.9426652606285209,
  'recall': 0.9836500313168568,
  'f1-score': 0.961280477976181,
  'support': 309317},
 'weighted avg': {'precision': 0.9769019406032929,
  'recall': 0.9739134932771235,
  'f1-score': 0.974490009291044,
  'support': 309317}}

### Serialise

In [708]:
import json
import pickle
import joblib

In [709]:
# serialise
with open('../deploy/columns.json', 'w') as fh:
    json.dump(X.columns.tolist(), fh)
    
with open('../deploy/dtypes.pickle', 'wb') as fh:
    pickle.dump(X.dtypes, fh)
    
joblib.dump(pipeline, '../deploy/pipeline.pickle')

PicklingError: Can't pickle <function fill_coordinates_with_station_means at 0x7f7eb6ed7d90>: it's not the same object as __main__.fill_coordinates_with_station_means